In [5]:
import pandas as pd
import numpy as np
df = pd.DataFrame({'A': [1, 2, 3], 'B': [4, 5, 6]})
print(df)
print()
print(type(df.mean()))
print()
df_mean = df.mean().to_dict()
print(df_mean)
print()
df_mean2 = df.mean().to_frame().T # 转为df
print(df_mean2)
print()
print(type(df['A'].mean())) # float64
print(type(df[['A']].mean())) # Series

print("mean(), sum(), std(), var(), min(), max(), median(), quantile(), skew(), kurt()返回的都是Series")
print("\n如果需要返回DataFrame，可以使用to_frame().T方法转换")
print(df.sum())
print()
print(df.std().to_frame().T)


   A  B
0  1  4
1  2  5
2  3  6

<class 'pandas.core.series.Series'>

{'A': 2.0, 'B': 5.0}

     A    B
0  2.0  5.0

<class 'numpy.float64'>
<class 'pandas.core.series.Series'>
mean(), sum(), std(), var(), min(), max(), median(), quantile(), skew(), kurt()返回的都是Series

如果需要返回DataFrame，可以使用to_frame().T方法转换
A     6
B    15
dtype: int64

     A    B
0  1.0  1.0


In [ ]:
df = pd.DataFrame({
    "A": ["foo", "bar", "foo", "bar", "foo", "bar", "foo", "foo"],
    "B": ["one", "one", "two", "three", "two", "two", "one", "three"],
    "C": np.random.randn(8),
    "D": np.random.randn(8),
})
print(df)
print()

# 按 A 分组，计算 C 和 D 的和
res1 = df.groupby("A")[["C", "D"]].sum()
print(res1)
print()
# 先按 A 分，A 相同的再按 B 分
res2 = df.groupby(["A", "B"]).sum()
print(res2)
print(df.groupby("A")[["C", "D"]].mean())

     A      B         C         D
0  foo    one  1.573346 -1.631746
1  bar    one -0.284010 -0.839970
2  foo    two -0.437080 -0.503197
3  bar  three  0.965461 -0.183623
4  foo    two  3.196080 -1.003276
5  bar    two  1.179924  1.588828
6  foo    one -1.594226 -0.235225
7  foo  three -0.577314  1.124380

            C         D
A                      
bar  1.861376  0.565235
foo  2.160807 -2.249064

                  C         D
A   B                        
bar one   -0.284010 -0.839970
    three  0.965461 -0.183623
    two    1.179924  1.588828
foo one   -0.020880 -1.866971
    three -0.577314  1.124380
    two    2.759000 -1.506473
            C         D
A                      
bar  0.620459  0.188412
foo  0.432161 -0.449813



In [13]:
# 多层（行)索引
arrays = [
    ["北京", "北京", "上海", "上海"],
    ["苹果", "香蕉", "苹果", "香蕉"]
]
index = pd.MultiIndex.from_arrays(arrays, names=["城市", "产品"])

# 创建数据：A、B 代表两个不同的年份
df = pd.DataFrame(np.random.randint(1, 10, (4, 2)), index=index, columns=["2025", "2026"])

print("原始数据 (宽格式):")
df

原始数据 (宽格式):


2025  2026
城市 产品            
北京 苹果     1     5
   香蕉     8     9
上海 苹果     6     7
   香蕉     5     8

In [ ]:
# 将列索引（2025, 2026）压入行索引 列变行 stack 表头变最内层行索引
# stack 堆叠 列变行 表格变成series 表格变瘦长
stacked = df.stack()

print("\nStack 之后 (长格式):\n")
print(stacked) # Series with MultiIndex 数据只有一列了


Stack 之后 (长格式):

城市  产品      
北京  苹果  2025    1
        2026    5
    香蕉  2025    8
        2026    9
上海  苹果  2025    6
        2026    7
    香蕉  2025    5
        2026    8
dtype: int32


In [ ]:
# unstack 行变列 默认把最后一层行索引（年份）弹回列（表头）
print("\nUnstack 恢复原状:")
print(stacked.unstack())


Unstack 恢复原状:
       2025  2026
城市 产品            
上海 苹果     6     7
   香蕉     5     8
北京 苹果     1     5
   香蕉     8     9


In [ ]:
# 索引编号从左往右是 0, 1, 2... 把第几层行索引弹到列 unstack
print("把 '城市' (第0层) 弹到列:")
print(stacked.unstack(0))

print("\n把 '产品' (第1层) 弹到列:")
print(stacked.unstack(1))

把 '城市' (第0层) 弹到列:
城市       上海  北京
产品             
苹果 2025   6   1
   2026   7   5
香蕉 2025   5   8
   2026   8   9

把 '产品' (第1层) 弹到列:
产品       苹果  香蕉
城市             
上海 2025   6   5
   2026   7   8
北京 2025   1   8
   2026   5   9


In [ ]:
# object 转 Categorical 映射存储 而非以重复文本存储 节省空间 
df = pd.DataFrame({
    "id": [1, 2, 3, 4, 5, 6], 
    "raw_grade": ["a", "b", "b", "a", "a", "e"]
})

print(df)
print()

df["grade"] = df["raw_grade"].astype("category")

print(df["grade"].dtype)
print()

# Rename categories
df["grade"] = df["grade"].cat.rename_categories({
    "a": "very good",
    "b": "good",
    "e": "poor"
})
print(df)
print()

# Reorder categories and add missing categories
# set categories 旧名字不在新的里面就变成 NaN 可以再 dropna()
# 新的有旧名字没有的-就产生空类别 value_counts 为 0
df["grade"] = df["grade"].cat.set_categories([
    "poor",
    "good",
    "very good",
    "excellent"
])
print(df) # 类别数量变成4

   id raw_grade
0   1         a
1   2         b
2   3         b
3   4         a
4   5         a
5   6         e

category

   id raw_grade      grade
0   1         a  very good
1   2         b       good
2   3         b       good
3   4         a  very good
4   5         a  very good
5   6         e       poor

   id raw_grade      grade
0   1         a  very good
1   2         b       good
2   3         b       good
3   4         a  very good
4   5         a  very good
5   6         e       poor


In [31]:
# 列表顺序 必须和原本 Categories 的顺序一致
# new_categories = ["very good", "good", "very bad"]
# df["grade"] = df["grade"].cat.rename_categories(new_categories)

logical_order = ["very bad", "bad", "medium", "good", "very good"]
df["grade"] = df["grade"].cat.set_categories(logical_order)
print(df) 
# 过滤数据 可以故意不把某些项写进 set_categories 变成NaN 然后dropna()
print()
print(df["grade"].cat.categories)

   id raw_grade      grade
0   1         a  very good
1   2         b       good
2   3         b       good
3   4         a  very good
4   5         a  very good
5   6         e        NaN

Index(['very bad', 'bad', 'medium', 'good', 'very good'], dtype='object')


In [39]:
# rename_categories 改名，比如poor -> bad
# set_categories 可以排序 或筛选、过滤不要的类别
df = pd.DataFrame({
    "id": [1, 2, 3, 4, 5, 6], 
    "raw_grade": ["a", "b", "b", "a", "a", "e"]
})

df["grade"] = df["raw_grade"].astype("category")

df["grade"] = df["grade"].cat.rename_categories({
    "a": "A",
    "b": "B",
    "e": "E"
})
print()
print(df)

# 规定以后只能有这 5 种类别，且按这个顺序排。数据里不在名单的都会变 NaN
df["grade"] = df["grade"].cat.set_categories(["E", "D", "C", "B", "A"], ordered=True)
print() # 别忘记赋值 不然df['grade']不变
print(df)

# 默认ordered=False 默认的 Categorical 是没有大小之分的 无法比较 没有max()
# ordered=True 按set_categories指定的顺序 如 E < D < C < B < A但未排序

# 排序：已指定顺序 E < D < C < B < A 
df_sorted = df.sort_values(by="grade")
# 默认 ascending = True
print("\n--- 排序后（按照你定的 E-D-C-B-A 顺序）---")
print(df_sorted)

# 5. 设置了 ordered=True之后，可以做数学比较
print("\n--- 验证大小比较 ---")
print(f"最高分是: {df['grade'].max()}") # 能算出 A 是最大值



   id raw_grade grade
0   1         a     A
1   2         b     B
2   3         b     B
3   4         a     A
4   5         a     A
5   6         e     E

   id raw_grade grade
0   1         a     A
1   2         b     B
2   3         b     B
3   4         a     A
4   5         a     A
5   6         e     E

--- 排序后（按照你定的 E-D-C-B-A 顺序）---
   id raw_grade grade
5   6         e     E
1   2         b     B
2   3         b     B
0   1         a     A
3   4         a     A
4   5         a     A

--- 验证大小比较 ---
最高分是: A


In [ ]:
# astype("category") 默认按照 字母升序 映射各类别为索引 0 1 2 如 a < b < e
# .cat 分类数据
df = pd.DataFrame({"raw_grade": ["a", "b", "e", "a"]})
df["grade"] = df["raw_grade"].astype("category") # 新列

print(df["grade"].cat.categories) 

df["grade"] = df["grade"].cat.rename_categories(
    ["优秀", "良好", "不及格"]) # 按照默认的字母升序 a b e分配新名字

logical_order = ["不及格", "及格", "良好", "优秀"]
df["grade"] = df["grade"].cat.set_categories(logical_order, ordered=True)
print(df)
# 增加了“及格”这个级别（尽管数据里还没人及格）
# 规定了 从小到大顺序

Index(['a', 'b', 'e'], dtype='object')
  raw_grade grade
0         a    优秀
1         b    良好
2         e   不及格
3         a    优秀


In [47]:
# 对 Series 使用 apply 会遍历每一个元素 df 各列为Series
df = pd.DataFrame({
    '姓名': ['张三', '李四', '王五'],
    '得分': [85, 92, 78]
})

def get_grade(score):
    if score >= 90:
        return '优秀'
    else:
        return '良好'

# 对 '得分' 这一列应用函数 新列
df['等级'] = df['得分'].apply(get_grade)

print(df)
# print(type(df['得分']))

# 需求：给姓名加上“先生/女士”后缀
df['姓名'] = df['姓名'].apply(lambda x: x + " 老师")
print()
print(df)

   姓名  得分  等级
0  张三  85  良好
1  李四  92  优秀
2  王五  78  良好

      姓名  得分  等级
0  张三 老师  85  良好
1  李四 老师  92  优秀
2  王五 老师  78  良好


In [53]:
# 对 df 使用 apply 
# 默认 axis = 0 对每列操作； axis = 1 对每行操作
df = pd.DataFrame({
    '语文': [80, 90, 70],
    '数学': [95, 85, 60]
})
print(df)
print()

# 对每行操作（求和）
#  x 代表“这一行”的数据对象
df['总分'] = df.apply(lambda x: x['语文'] + x['数学'], axis=1)
print(df)

print()
# 每一列的最大值与最小值的差 x是每一列
result = df.apply(lambda x: x.max() - x.min(), axis=0)
print(result) # Series

# 对每个格子操作 用 applymap
# 需求：将所有数值保留两位小数，并转为字符串格式
df_formatted = df.applymap(lambda x: f"{x:.2f}")
print()
print(df_formatted)

   语文  数学
0  80  95
1  90  85
2  70  60

   语文  数学   总分
0  80  95  175
1  90  85  175
2  70  60  130

语文    20
数学    35
总分    45
dtype: int64

      语文     数学      总分
0  80.00  95.00  175.00
1  90.00  85.00  175.00
2  70.00  60.00  130.00


C:\Users\19588\AppData\Local\Temp\ipykernel_6356\2517447472.py:22: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_formatted = df.applymap(lambda x: f"{x:.2f}")


In [ ]:
# 对Series 用map
# 字典映射
df = pd.DataFrame({
    '姓名': ['小明', '小红', '小刚'],
    '性别': ['男', '女', '男'],
    '等级': ['A', 'B', 'A']
})

gender_dict = {'男': 1, '女': 0}

# 使用 map 映射
df['性别编号'] = df['性别'].map(gender_dict)

print(df)
print()

# 如果字典里没有涵盖某个值，map 会把该值变成 NaN
data = pd.Series(['猫', '狗', '兔子'])
mapping = {'猫': '喵喵', '狗': '汪汪'}
result = data.map(mapping)
print(result)
print()

df = pd.DataFrame({'比例': [0.8852, 0.1234, 0.9]})

# 0.8852 -> 88.5% map 对这列每个数字操作
df['格式化比例'] = df['比例'].map("{:.1%}".format)

print(df)

# 字符串方法 .format()
val = 0.8852
formatted_val = "{:.1%}".format(val)  # 结果: "88.5%"

   姓名 性别 等级  性别编号
0  小明  男  A     1
1  小红  女  B     0
2  小刚  男  A     1

0     喵喵
1     汪汪
2    NaN
dtype: object

       比例  格式化比例
0  0.8852  88.5%
1  0.1234  12.3%
2  0.9000  90.0%
